<a href="https://colab.research.google.com/github/dt2229/sandp500/blob/main/DB_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('white')

import statsmodels.api as sm
import statsmodels.tsa.stattools as ts

from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import time

In [2]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 773 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [3]:
import yfinance

DB = yfinance.download (tickers = "DB", start = "2012-01-03", 
                              end = "2022-10-15", interval = "1d")

[*********************100%***********************]  1 of 1 completed


In [4]:
DB.reset_index(inplace = True)
DB['Date'] = DB['Date'].apply(lambda x: str(x)[:10])
DB.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2012-01-03,37.853054,38.349236,37.738548,38.005726,32.415062,2675544
1,2012-01-04,36.784351,37.013359,36.202290,36.860687,31.438459,2407256
2,2012-01-05,34.351147,34.856869,33.921757,34.570610,29.485252,4930421
3,2012-01-06,33.912212,33.912212,32.652672,32.690842,27.882000,4366178
4,2012-01-09,32.423664,32.461830,31.765266,32.099236,27.377419,5461966


In [5]:
DB.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
DB['ret'] = DB['Adj Close'].pct_change()
data = DB[['Date', 'Adj Close']]
data.head()

,Date,Adj Close
0,2012-01-03,32.415062
1,2012-01-04,31.438459
2,2012-01-05,29.485252
3,2012-01-06,27.882000
4,2012-01-09,27.377419


In [6]:
data.set_index('Date', inplace = True)
import math

In [7]:
close_prices = data['Adj Close']
values = close_prices.values
training_data_len = math.ceil(len(values)* 0.8)

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(values.reshape(-1,1))
train_data = scaled_data[0: training_data_len, :]

x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
    
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [8]:

test_data = scaled_data[training_data_len-60: , : ]
x_test = []
y_test = values[training_data_len:]

for i in range(60, len(test_data)):
  x_test.append(test_data[i-60:i, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [9]:
from sklearn.model_selection import TimeSeriesSplit
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

In [10]:
tscv = TimeSeriesSplit(max_train_size=None, n_splits=3)
print(tscv)

mape_lst = []

for train_index, valid_index in tscv.split(x_train):
  X_train, X_valid = x_train[train_index], x_train[valid_index]
  Y_train, Y_valid = y_train[train_index], y_train[valid_index]
  model = keras.Sequential()
  model.add(layers.LSTM(100, return_sequences=True, input_shape=(x_train.shape[1], 1)))
  model.add(layers.LSTM(100, return_sequences=False))
  model.add(layers.Dense(25))
  model.add(layers.Dense(1))
  model.summary()
  model.compile(optimizer='adam', loss='mean_squared_error')
  model.fit(X_train, Y_train, batch_size= 64, epochs=3)
  predictions = model.predict(X_valid)
  #predictions = scaler.inverse_transform(predictions)
  metrics = mape(predictions, Y_valid)
  print(metrics)
  mape_lst.append(metrics)

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 100)           40800     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 25)                2525      
                                                                 
 dense_1 (Dense)             (None, 1)                 26        
                                                                 
Total params: 123,751
Trainable params: 123,751
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
9/9 [==============================] - 12s 305ms/step - loss: 0.1769
Epoch 2/3
9/9 [================

In [11]:
model = keras.Sequential()
model.add(layers.LSTM(100, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(layers.LSTM(100, return_sequences=False))
model.add(layers.Dense(25))
model.add(layers.Dense(1))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 60, 100)           40800     
                                                                 
 lstm_7 (LSTM)               (None, 100)               80400     
                                                                 
 dense_6 (Dense)             (None, 25)                2525      
                                                                 
 dense_7 (Dense)             (None, 1)                 26        
                                                                 
Total params: 123,751
Trainable params: 123,751
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, batch_size= 1, epochs=3)

Epoch 1/3
2112/2112 [==============================] - 78s 35ms/step - loss: 0.0018
Epoch 2/3
2112/2112 [==============================] - 69s 32ms/step - loss: 6.9774e-04
Epoch 3/3
2112/2112 [==============================] - 68s 32ms/step - loss: 5.8478e-04


In [13]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

17/17 [==============================] - 1s 27ms/step


In [14]:
mape(predictions, y_test)

19.05942529813318